<a href="https://colab.research.google.com/github/osgeokr/GEE-PAM-Book/blob/main/Google_Earth_Engine_Maxent_%EA%B5%AC%ED%98%84_%EA%B8%B0%EB%8A%A5_%EC%86%8C%EA%B0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee
import geemap

# Earth Engine 인증
ee.Authenticate()

# Earth Engine 초기화
ee.Initialize(project='ee-foss4g')

In [2]:
# Create some sample species presence/absence training data.

# 출현/부재 훈련 데이터 생성
training_data = ee.FeatureCollection([
    # 종 출현 지점
    ee.Feature(ee.Geometry.Point([-122.39567, 38.02740]), {'presence': 1}),
    ee.Feature(ee.Geometry.Point([-122.68560, 37.83690]), {'presence': 1}),
    # 종 부재 지점
    ee.Feature(ee.Geometry.Point([-122.59755, 37.92402]), {'presence': 0}),
    ee.Feature(ee.Geometry.Point([-122.47137, 37.99291]), {'presence': 0}),
    ee.Feature(ee.Geometry.Point([-122.52905, 37.85642]), {'presence': 0}),
    ee.Feature(ee.Geometry.Point([-122.03010, 37.66660]), {'presence': 0})
])

In [3]:
# Landsat 8 이미지를 가져와 반사율 밴드 선택
image = (ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_044034_20200606')
         .select(['SR_B[1-7]'])
         .multiply(0.0000275).add(-0.2))  # 축척 계수 적용

In [4]:
# 종 출현/부재 위치에서 이미지 샘플링
training = image.sampleRegions(**{
    'collection': training_data,
    'scale': 30
})

In [5]:
# 이미지 샘플링된 지점에서 Maxent 분류기 정의 및 훈련
classifier = ee.Classifier.amnhMaxent().train(**{
    'features': training,
    'classProperty': 'presence',
    'inputProperties': image.bandNames()
})

In [6]:
# Maxent 분류기를 사용하여 이미지 분류
image_classified = image.classify(classifier)

In [8]:
# 지도 생성
m = geemap.Map()

# 이미지를 기반으로 지도 중심을 설정하고 줌 레벨을 설정합니다.
m.centerObject(image, 9)

# 지도에 원본 이미지 레이어를 표시합니다.
# 시각화 파라미터에 맞춰 스케일링을 적용합니다.
m.addLayer(
    image.select(['SR_B4', 'SR_B3', 'SR_B2']).multiply(0.0000275).add(-0.2),
    {'min': 0, 'max': 0.3}, 'Image'
)

# 분류된 확률 레이어를 표시합니다.
m.addLayer(
    image_classified.select('probability'), {'min': 0, 'max': 1, 'palette': ['black', 'white']},
    'Probability'
)

# 종이 없는 위치(빨강)와 있는 위치(파랑)에서 훈련 데이터 포인트를 표시합니다.
m.addLayer(
    training_data.filter(ee.Filter.eq('presence', 0)), {'color': 'red'},
    '훈련 데이터 (종 부재)'
)
m.addLayer(
    training_data.filter(ee.Filter.eq('presence', 1)), {'color': 'blue'},
    '훈련 데이터 (종 출현)'
)

# 지도를 표시합니다.
m

Map(center=[37.4729221349684, -122.10482352567668], controls=(WidgetControl(options=['position', 'transparent_…